This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [2]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)
#Key is correct so don't print

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [14]:
# First, import the relevant modules
import json
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [31]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
base_url = 'https://data.nasdaq.com/api/v3/'
data_base = 'datasets/FSE/'
data_set_code = 'AFX_X/'
data_type = 'json'
params_test = {'start_date': '2014-01-01', 
        'end_date': '2014-01-03',
        'api_key': API_KEY
         }

url_test = base_url + data_base + data_set_code + 'data.' + data_type 
r_test = requests.get(url, params = params_test)

In [29]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r_test.status_code

200

In [32]:
r_test.content

b'{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2014-01-01","end_date":"2014-01-03","frequency":"daily","data":[["2014-01-03",24.04,24.47,24.03,24.47,null,20503.0,499475.0,null,null,null],["2014-01-02",24.24,24.76,23.77,24.18,null,55735.0,1355658.0,null,null,null]],"collapse":null,"order":null}}'

In [36]:
test_dict = json.loads(r_test.content.decode('utf-8'))

In [75]:
test_dict['dataset_data']

{'limit': None,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'start_date': '2014-01-01',
 'end_date': '2014-01-03',
 'frequency': 'daily',
 'data': [['2014-01-03',
   24.04,
   24.47,
   24.03,
   24.47,
   None,
   20503.0,
   499475.0,
   None,
   None,
   None],
  ['2014-01-02',
   24.24,
   24.76,
   23.77,
   24.18,
   None,
   55735.0,
   1355658.0,
   None,
   None,
   None]],
 'collapse': None,
 'order': None}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [33]:
base_url = 'https://data.nasdaq.com/api/v3/'
data_base = 'datasets/FSE/'
data_set_code = 'AFX_X/'
data_type = 'json'
params = {'start_date': '2017-01-01', 
        'end_date': '2017-12-31',
        'api_key': API_KEY
         }

url = base_url + data_base + data_set_code + 'data.' + data_type 

In [34]:
#1. The data is collected into the variable r
r = requests.get(url, params = params)

In [49]:
#2. Data is converted to a pythonn dictionary
data = json.loads(r.content.decode('utf-8'))['dataset_data']

In [70]:
#3
def get_col(col_name):
    ind = data['column_names'].index(col_name)
    return [row[ind] for row in data['data'] if row[ind]]

opens = get_col('Open')
print('The highest opening price was', max(opens))
print('The lowest opening price was', min(opens))

The highest opening price was 53.11
The lowest opening price was 34.0


In [57]:
#4
high_ind = data['column_names'].index('High') 
low_ind = data['column_names'].index('Low')
changes = [row[high_ind]-row[low_ind] for row in data['data'] if row[high_ind] and row[low_ind]]

print('The largest change in one day is', max(changes))

The largest change in one day is 2.8100000000000023


In [71]:
#5
closes = get_col('Close')
print('The largest difference in closing price between any two days is', max(closes)-min(closes))

The largest difference in closing price between any two days is 19.03


In [73]:
#6 
vols = get_col('Traded Volume')
print('The average traded volume daily is', sum(vols)/len(vols))

The average traded volume daily is 89124.33725490196


In [74]:
#7 
vols_sort = sorted(vols)
n = len(vols_sort)

if n%2 == 1:
    vol_med = vols_sort[n//2]
else:
    vol_med = (vols_sort[n//2] + vols_sort[n//2-1])/2

print('The medium traded volume daily is', vol_med)

The medium traded volume daily is 76286.0
